# LinkedIn Job Postings

## ADS 508 Impacting the Business with a Distributed Data Science Pipeline

In [3]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import sagemaker
from pyathena import connect
import awswrangler as wr
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
from io import StringIO
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

import warnings
warnings.filterwarnings('ignore')

## Data Ingestion

### CSV to S3

In [4]:
# Setup boto3 session parameters
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Establish connection
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [5]:
# Set S3 Source Location (Public bucket)
s3_public_path = "s3://linkedin-postings"

In [ ]:
%store s3_public_path

In [ ]:
# Set S3 Destination Location (Private bucket)
s3_private_path = "s3://{}/linkedin_data".format(bucket)
print(s3_private_path)

In [ ]:
%store s3_private_path

In [ ]:
# Copy data from Public S3 bucket to Private S3 bucket
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "postings/postings.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "salaries/salaries.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "job_skills/job_skills.csv"

In [ ]:
# Check files are copied successfully to private bucket
!aws s3 ls $s3_private_path/

In [ ]:
# Initialize boto3 client
s3 = boto3.client('s3')

# Define bucket and paths 
bucket_name = bucket
file_key = 'linkedin_data/postings/postings.csv'
cleaned_file_key = 'linkedin_data/postings/cleaned/cleaned_postings.csv'

# Read postings.csv directly from private bucket
obj = s3.get_object(Bucket=bucket_name, Key=file_key)
df = pd.read_csv(obj['Body'])

# Remove embedded newlines
df['description'].replace({r'[\n\r]+': ' '}, regex=True, inplace=True)
df['skills_desc'].replace({r'[\n\r]+': ' '}, regex=True, inplace=True)

# Remove embedded commas
df['company_name'].replace({r'[,]+': ' '}, regex=True, inplace=True)
df['title'].replace({r'[,]+': ' '}, regex=True, inplace=True)
df['description'].replace({r'[,]+': ' '}, regex=True, inplace=True)
df['location'].replace({r'[,]+': ' '}, regex=True, inplace=True)
df['skills_desc'].replace({r'[,]+': ' '}, regex=True, inplace=True)


# Save cleaned CSV back to S3 directly (in-memory)
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

s3.put_object(Bucket=bucket_name, Key=cleaned_file_key, Body=csv_buffer.getvalue())

print(f"Cleaned CSV successfully uploaded to: s3://{bucket_name}/{cleaned_file_key}")

### Create Athena Database 

In [13]:
ingest_create_athena_db_passed = False

In [14]:
ingest_create_athena_table_passed = False

In [15]:
database_name = "linkedin_data"

In [16]:
# Set S3 staging directory - a temporary directory for Athena queries
s3_staging_dir = "s3://{}/athena/staging".format(bucket)

In [17]:
# Connect to staging directory
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [ ]:
# Create Database
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

pd.read_sql(statement, conn)

### Verify database has been created

In [ ]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

In [20]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

### Create Athena Tables from CSV Files

In [ ]:
table_name = 'postings'
postings_path = "s3://{}/linkedin_data/postings/cleaned/".format(bucket)

drop_statement = """DROP TABLE IF EXISTS {}.{};""".format(database_name, table_name)

print(drop_statement)
pd.read_sql(drop_statement, conn)
print("Attempted to Drop {} table".format(table_name))

In [ ]:
# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        job_id string,
        company_name string,
        title string,
        description string,
        max_salary float,
        pay_period string,
        location string,
        company_id float,
        views float,
        med_salary float,
        min_salary float,
        formatted_work_type string,
        applies float,
        original_listed_time float,
        remote_allowed float,
        job_posting_url string,
        application_url string,
        application_type string,
        expiry float,
        closed_time float,
        formatted_experience_level string,
        skills_desc string,
        listed_time string,
        posting_domain string,
        sponsored int,
        work_type string,
        currency string,
        compensation_type string,
        normalized_salary float,
        zip_code int,
        fips int
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name, postings_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created postings table")

In [ ]:
table_name_2 = "salaries"
salaries_path = "s3://{}/linkedin_data/salaries/".format(bucket)

drop_statement2 = """DROP TABLE IF EXISTS {}.{};""".format(database_name, table_name_2)

print(drop_statement2)
pd.read_sql(drop_statement2, conn)
print("Attempted to Drop {} table".format(table_name_2))

In [ ]:
# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        salary_id int,
        job_id string,
        max_salary float,
        med_salary float,
        min_salary float,
        pay_period string,
        currency string,
        compensation_type string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name_2, salaries_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created salaries table")

In [ ]:
table_name_3 = "job_skills"
job_skills_path = "s3://{}/linkedin_data/job_skills/".format(bucket)

drop_statement3 = """DROP TABLE IF EXISTS {}.{};""".format(database_name, table_name_3)

print(drop_statement3)
pd.read_sql(drop_statement3, conn)
print("Attempted to Drop {} table".format(table_name_3))

In [ ]:
# SQL statement to execute the postings table
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        job_id string,
        skill_abr string
    ) 
    ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
    LOCATION '{}' 
    TBLPROPERTIES ('skip.header.line.count'='1')
    """.format(database_name, table_name_3, job_skills_path)

# Execute statement
pd.read_sql(statement, conn)
print("Created job_skills table")

## Create Athena Parquet Tables

### Postings Parquet Table

In [ ]:
table_name = 'postings_parquet'
drop_statement = """DROP TABLE IF EXISTS {}.{};""".format(database_name, table_name)

print(drop_statement)
pd.read_sql(drop_statement, conn)
print("Attempted to Drop {} table".format(table_name))

In [ ]:
table_name = "postings"
table_name_parquet = "postings_parquet"
postings_parquet_path = "s3://{}/linkedin_data/parquet/postings/".format(bucket)

# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}') AS
SELECT job_id,
        company_name,
        title,
        description,
        max_salary,
        pay_period,
        location,
        company_id,
        views,
        med_salary,
        min_salary,
        formatted_work_type,
        applies,
        original_listed_time,
        remote_allowed,
        job_posting_url,
        application_url,
        application_type,
        expiry,
        closed_time,
        formatted_experience_level,
        skills_desc,
        listed_time,
        posting_domain,
        sponsored,
        work_type,
        currency,
        compensation_type,
        normalized_salary,
        zip_code,
        fips
FROM {}.{}""".format(
    database_name, table_name_parquet, postings_parquet_path, database_name, table_name
)

pd.read_sql(statement, conn)

### Salaries Parquet table

In [ ]:
table_name = "salaries"
table_name_parquet = "salaries_parquet"
salaries_parquet_path = "s3://{}/linkedin_data/parquet/salaries/".format(bucket)

# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}') AS
SELECT salary_id,
    job_id,
    max_salary,
    med_salary,
    min_salary,
    pay_period,
    currency,
    compensation_type
FROM {}.{}""".format(
    database_name, table_name_parquet, salaries_parquet_path, database_name, table_name
)


pd.read_sql(statement, conn)

### Job Skills Parquet table

In [ ]:
table_name = "job_skills"
table_name_parquet = "job_skills_parquet"
job_skills_parquet_path = "s3://{}/linkedin_data/parquet/job_skills/".format(bucket)

# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}') AS
SELECT job_id,
        skill_abr
FROM {}.{}""".format(
    database_name, table_name_parquet, job_skills_parquet_path, database_name, table_name
)

pd.read_sql(statement, conn)

### Verify tables have been created successfully

In [ ]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(10)

In [32]:
if table_name in df_show.values:
    ingest_create_athena_table_passed = True

In [ ]:
# View postings table to check the data looks correct
statement = """SELECT * FROM {}.{} LIMIT 5""".format(database_name, 'postings_parquet')

pd.read_sql(statement, conn)

In [ ]:
# View salaries table to check the data looks correct
statement = """SELECT * FROM {}.{} LIMIT 5""".format(database_name, 'salaries_parquet')

pd.read_sql(statement, conn)

In [ ]:
# View job_skills table to check the data looks correct
statement = """SELECT * FROM {}.{} LIMIT 5""".format(database_name, 'job_skills_parquet')

pd.read_sql(statement, conn)

## Data Exploration

In [ ]:
postings_df = wr.athena.read_sql_query("""SELECT * FROM postings_parquet""", database=database_name)

In [37]:
salaries_df = wr.athena.read_sql_query("""SELECT * FROM salaries_parquet""", database=database_name)

In [38]:
job_skills_df = wr.athena.read_sql_query("""SELECT * FROM job_skills_parquet""", database=database_name)

### postings EDA

In [ ]:
# Basic info
postings_df.info()

In [ ]:
# Missing values
postings_df.isnull().sum()

In [ ]:
# Duplicates
duplicates = postings_df.duplicated().sum()
print(duplicates)

In [ ]:
# Summary statistics for numerical columns
print(postings_df.describe())

In [ ]:
# Top 10 unique values in categorical columns
categorical = ["title", "zip_code", "formatted_work_type", "remote_allowed"]
for col in categorical:
    print(postings_df[col].value_counts().head(10))

In [ ]:
# Detect Outliers using box plots
numerical = ["views", "applies"]
for col in numerical:
    plt.figure(figsize=(5, 2))
    sns.boxplot(x=postings_df[col])
    plt.title(f"Outlier Detection - {col}")
    plt.show()

In [ ]:
# Visualize experience level distribution
top_titles = postings_df["formatted_experience_level"].value_counts().iloc[[1,3,7]]

plt.figure(figsize=(10,5))
sns.barplot(y=top_titles.index, x=top_titles.values, palette="coolwarm")
plt.xlabel("Number of Postings")
plt.ylabel("Experience Level")
plt.title("Job Postings by Experience Level")
plt.xticks(rotation=45)
plt.show()

### salaries EDA

In [ ]:
# Basic stats
print(salaries_df.describe())
print(salaries_df.info())

In [ ]:
# Missing values
salaries_df.isnull().sum()

In [ ]:
# Duplicates
salaries_df.duplicated().sum()

In [ ]:
# Detect Outliers using box plots
numerical = ["max_salary", "med_salary", "min_salary"]
for col in numerical:
    plt.figure(figsize=(5, 2))
    sns.boxplot(x=postings_df[col])
    plt.title(f"Outlier Detection - {col}")
    plt.show()

In [ ]:
# Salary distributions
salary_columns = ["min_salary", "med_salary", "max_salary"]
print(salaries_df[salary_columns].describe())

In [ ]:
salaries_df.hist(figsize=(18,12))

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(salaries_df["min_salary"], color="blue", kde=True, label="Min Salary", bins=30)

# Customize plot
plt.title("Salary Distribution (Histogram)")
plt.xlabel("Salary")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(salaries_df["med_salary"], color="green", kde=True, label="Median Salary", bins=30)

# Customize plot
plt.title("Salary Distribution (Histogram)")
plt.xlabel("Salary")
plt.ylabel("Frequency")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(salaries_df["max_salary"], color="red", kde=True, label="Max Salary", bins=30)

# Customize plot
plt.title("Salary Distribution (Histogram)")
plt.xlabel("Salary")
plt.ylabel("Frequency")
plt.legend()
plt.show()

### job_skills EDA

In [ ]:
# Basic info
print(job_skills_df.describe())
print(job_skills_df.info())

In [ ]:
# Missing values
job_skills_df.isnull().sum()

In [ ]:
# Duplicates
job_skills_df.duplicated().sum()

In [ ]:
# Unique skills
print(f"Unique Job Skills: {job_skills_df['skill_abr'].nunique()}")

In [ ]:
# Skill frequencies
job_skills_df["skill_abr"] = job_skills_df["skill_abr"].astype(str)  # Ensure it's a string
skills = job_skills_df["skill_abr"].str.split(",").explode().str.strip().value_counts()
print(skills.head(20))

In [ ]:
# Concatenate all skills descriptions
all_skills = " ".join(job_skills_df["skill_abr"].dropna())

# Generate WordCloud
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(all_skills)

# Bar Chart of Top 20 Most Common Skills
top_skills = skills.head(20)

plt.figure(figsize=(10, 5))
sns.barplot(y=top_skills.index, x=top_skills.values, palette="viridis")
plt.title("Top 20 Most Common Skills")
plt.xlabel("Number of Occurrences")
plt.ylabel("Skill")
plt.show()

### Pre-Processing

In [ ]:
# Missing values - postings_df
postings_df.isnull().sum()

In [ ]:
# Missing values - salaries_df
salaries_df.isnull().sum()

In [ ]:
# Missing values - job_skills_df
job_skills_df.isnull().sum()

In [64]:
# Remove unnecessary columns
columns_to_keep = ['job_id', 'title', 'pay_period', 'remote_allowed', 'formatted_work_type', 'zip_code']
postings_df = postings_df[columns_to_keep]

In [ ]:
postings_df.head()

In [68]:
# Remove missing values in pay_period column
postings_df = postings_df.dropna(subset=['pay_period'])

In [69]:
# Fill in empty cells with 0 for remote NOT allowed
postings_df['remote_allowed'].fillna(0, inplace=True)

In [71]:
# Removed .0 at the end of all zipcodes and filled NaN with 0
postings_df['zip_code'] = postings_df['zip_code'].fillna(0).astype(int).astype(str)

In [ ]:
# Double check df looks right
postings_df.info()
postings_df.isna().sum()

In [73]:
# Fill NaN with median of respective column
salaries_df['max_salary'].fillna(salaries_df['max_salary'].median(), inplace=True)
salaries_df['min_salary'].fillna(salaries_df['min_salary'].median(), inplace=True)
salaries_df['med_salary'].fillna(salaries_df['med_salary'].median(), inplace=True)

In [74]:
# Ensure there are no incorrect data types for salaries
salaries_df['max_salary'] = pd.to_numeric(salaries_df['max_salary'], errors='coerce')
salaries_df['min_salary'] = pd.to_numeric(salaries_df['min_salary'], errors='coerce')
salaries_df['med_salary'] = pd.to_numeric(salaries_df['med_salary'], errors='coerce')

In [ ]:
# write pre-processed df to S3
csv_buffer = StringIO()
postings_df.to_csv(csv_buffer, index=False)
file_key = 'linkedin_data/postings/preprocessing/postings.csv'

s3.put_object(Bucket=bucket, Key=file_key, Body=csv_buffer.getvalue())

## Data Transformations

### Create embeddings for title

In [76]:
# create the processor instance
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0",
    role=role,
    instance_type="ml.m5.xlarge",
    instance_count=1,
)

In [ ]:
# check the input data
key_file = 'linkedin_data/postings/preprocessing/postings.csv'
input_data = 's3://{}/{}'.format(bucket, key_file)
obj = s3.get_object(Bucket=bucket, Key=key_file)
df = pd.read_csv(obj['Body'], nrows=10)
df.head(n=10)

### Define the processing job

In [ ]:
%%writefile preprocessing.py 
import os

os.system('pip install sentence-transformers')

if __name__ == '__main__':
    from sentence_transformers import SentenceTransformer
    from sklearn.decomposition import PCA
    
    categorical_columns = ['zip_code', 'formatted_work_type', 'remote_allowed', 'pay_period']
    
    model = SentenceTransformer("all-MiniLM-L6-v2") # SBERT, not case-sensitive

    input_data_path = os.path.join('/opt/ml/processing/input', 'postings.csv')

    print("Reading input data from {}".format(input_data_path))
    df = pd.read_csv(input_data_path)
    
    # create embeddings for 'title'
    print('Creating embeddings for job title')
    df["title_embeddings"] = df["title"].apply(lambda x: model.encode(x))

    # Use PCA to reduce the embeddings vector down to 2 dimensions 
    print('Reducing embeddings to 2 dimensions')
    pca = PCA(n_components=2, svd_solver='full')
    embeddings_matrix = np.vstack(df['title_embeddings'].values)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    
    df['pca_1'] = reduced_embeddings[:, 0]
    df['pca_2'] = reduced_embeddings[:, 1]

    # One hot encode categorical fields
    df = pd.get_dummies(df, columns=categorical_columns)
    
    # save csv to output_path
    output_path = os.path.join('/opt/ml/processing/output', 'postings.csv')
    df.to_csv(output_path, index=False)

### Execute the processing job

In [ ]:
# run the processing job
sklearn_processor.run(
    code="preprocessing.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="processed_postings", source="/opt/ml/processing/output")
    ]
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "processed_postings":
        preprocessed_data = output["S3Output"]["S3Uri"]
        
        # check output of processing job
        df = pd.read_csv(preprocessed_data + "/postings.csv", nrows=10)
        df.head(n=10)

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: 80% train, 20% temp
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42)

# Step 2: Split the remaining 20% evenly
df_validate, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

# Confirm the splits
print("Train shape:", df_train.shape)
print("Validate shape:", df_validate.shape)
print("Test shape:", df_test.shape)


In [ ]:
import boto3
from io import StringIO

# Use your private bucket from earlier
s3 = boto3.client('s3')
bucket_name = bucket  # this should already be set
base_path = 'linkedin_data/partition'

# Upload helper function
def upload_df_to_s3(df, key):
    buffer = StringIO()
    df.to_csv(buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=key, Body=buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket_name}/{key}")

# Upload each partition
upload_df_to_s3(df_train, f'{base_path}/train/train.csv')
upload_df_to_s3(df_validate, f'{base_path}/validate/validate.csv')
upload_df_to_s3(df_test, f'{base_path}/test/test.csv')


### Model Training

In [ ]:
# Load from S3 into pandas
s3 = boto3.client('s3')
obj = s3.get_object(
    Bucket= session.default_bucket(),
    Key='linkedin_data/partition/train/train.csv'
)
df = pd.read_csv(obj['Body'])

# Convert to NumPy
train_np = df.to_numpy()

# Record shape
num_records = train_np.shape[0]
feature_dim = train_np.shape[1]

print("Training shape:", train_np.shape)


In [ ]:
# Your preprocessing
df_numeric = df.select_dtypes(include=['int64', 'float64', 'bool'])
train_np = df_numeric.astype('float32').to_numpy()

# Insert this
from sagemaker.image_uris import retrieve
from sagemaker import KMeans

container = retrieve('kmeans', region=region)

kmeans = KMeans(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    k=5,
    output_path=f's3://{bucket}/output/kmeans/',
    sagemaker_session=session,
    image_uri=container,
    mini_batch_size=max(500, int(num_records * 0.05))
)

# Now this will work
train_data = kmeans.record_set(train_np)
kmeans.fit(train_data)


In [ ]:
# Deploy the trained KMeans model
kmeans_predictor = kmeans.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large'
)


In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Set serialization for inference
kmeans_predictor.serializer = CSVSerializer()
kmeans_predictor.deserializer = JSONDeserializer()

def predict_in_batches(predictor, data, batch_size=1000):
    predictions = []

    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        result = predictor.predict(batch)
        batch_preds = [int(pred['closest_cluster']) for pred in result['predictions']]
        predictions.extend(batch_preds)

    return predictions


In [ ]:
# Predict cluster assignments in batches
assignments = predict_in_batches(kmeans_predictor, input_data)

# Add predictions to DataFrame
df_with_clusters = df.copy()
df_with_clusters['cluster'] = assignments

df_with_clusters.head()


In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    df_with_clusters['pca_1'], 
    df_with_clusters['pca_2'], 
    c=df_with_clusters['cluster'], 
    cmap='tab10', 
    alpha=0.6
)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('KMeans Clusters on Job Postings')
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.show()


In [ ]:
# Load test set
obj_test = s3.get_object(
    Bucket= session.default_bucket(),
    Key='linkedin_data/partition/test/test.csv'
)
df_test = pd.read_csv(obj_test['Body'])

# Load validation set
obj_val = s3.get_object(
    Bucket= session.default_bucket(),
    Key='linkedin_data/partition/validate/validate.csv'
)
df_val = pd.read_csv(obj_val['Body'])


In [ ]:
# Make sure df_numeric is already defined from training
# df_numeric = df.select_dtypes(include=['int64', 'float64', 'bool'])

# Keep only numeric features and convert to float32
df_test_numeric = df_test[df_numeric.columns].astype('float32')
df_val_numeric = df_val[df_numeric.columns].astype('float32')

# Convert to NumPy arrays
test_input = df_test_numeric.to_numpy()
val_input = df_val_numeric.to_numpy()

# Sanity check
print("Test shape:", test_input.shape, "| dtype:", test_input.dtype)
print("Val shape:", val_input.shape, "| dtype:", val_input.dtype)


In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

kmeans_predictor.serializer = CSVSerializer()
kmeans_predictor.deserializer = JSONDeserializer()


In [ ]:
def predict_in_batches(predictor, data, batch_size=1000):
    predictions = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        result = predictor.predict(batch)
        batch_preds = [int(pred['closest_cluster']) for pred in result['predictions']]
        predictions.extend(batch_preds)
    return predictions


In [ ]:
test_assignments = predict_in_batches(kmeans_predictor, test_input, batch_size=250)
val_assignments = predict_in_batches(kmeans_predictor, val_input, batch_size=250)


In [ ]:
df_test['cluster'] = test_assignments

df_test.head()

In [ ]:
df_val['cluster'] = val_assignments
df_val.head()

In [ ]:
def upload_to_s3(df, key):
    buffer = StringIO()
    df.to_csv(buffer, index=False)
    s3.put_object(
        Bucket= session.default_bucket(),
        Key=key,
        Body=buffer.getvalue()
    )
    print(f"✅ Uploaded to s3://private-bucket/{key}")

upload_to_s3(df_test, 'linkedin_data/partitions/predictions/test_with_clusters.csv')
upload_to_s3(df_val, 'linkedin_data/partitions/predictions/val_with_clusters.csv')


In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    df_test['pca_1'],
    df_test['pca_2'],
    c=df_test['cluster'],
    cmap='tab10',
    alpha=0.6
)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('KMeans Clusters - Test Set')
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    df_val['pca_1'],
    df_val['pca_2'],
    c=df_val['cluster'],
    cmap='tab10',
    alpha=0.6
)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('KMeans Clusters - Validation Set')
plt.colorbar(scatter, label='Cluster')
plt.grid(True)
plt.show()


In [ ]:
### Evaluating With Silhouette 

In [ ]:
from sklearn.metrics import silhouette_score

# Make sure only using numeric columns used in clustering
X_train = df_with_clusters[df_numeric.columns]
X_test = df_test[df_numeric.columns]
X_val = df_val[df_numeric.columns]

# Get the cluster assignments
labels_train = df_with_clusters['cluster']
labels_test = df_test['cluster']
labels_val = df_val['cluster']

# Calculate silhouette scores
score_train = silhouette_score(X_train, labels_train)
score_test = silhouette_score(X_test, labels_test)
score_val = silhouette_score(X_val, labels_val)

print("Silhouette Score (Train):", round(score_train, 4))
print("Silhouette Score (Test):", round(score_test, 4))
print("Silhouette Score (Validation):", round(score_val, 4))
